In [4]:
import pandas
import random
import os
import json

## Part 1

### Customer

In [5]:
custFileName = "nameList1000.txt"

ROOT = os.path.dirname(os.getcwd()) + '\\resource\\' + custFileName

first_names = []
last_names = []

with open(ROOT, 'r') as file:
    rawnames = file.read().split('\n')

for name in rawnames:
    first_names.append(name[:name.find(' ')])
    last_names.append(name[1+name.find(' '):])

In [6]:
'''
--
-- given list of 1st and 2nd names
-- randomize them and duplicate them randomly for repeat customers
--
'''

# Create a list to hold all names
names = []

# Generate unique names (50% of 4000 = 2000 unique names)
for index in range(2000):
    first = random.choice(first_names)
    last = random.choice(last_names)
    name = f"{first} {last}"
    names.append(name)

# Generate repeated names (remaining 50% will be repeats 5-10 times)
for index in range(1000):  # Need 1000 more distinct names, each repeating 5-10 times
    first = random.choice(first_names)
    last = random.choice(last_names)
    name = f"{first} {last}"
    repeat_count = random.randint(5, 10)  # Repeat each name 5 to 10 times
    names.extend([name] * repeat_count)

# Shuffle the final list to mix unique and repeated names
random.shuffle(names)

# Ensure the list is exactly 4000 names
names = names[:25]

# Convert the list to a single line with commas separating each name
# result = ", ".join(names)

In [7]:
'''
--
-- DF to pandas
--
'''
customerDict = {}
first_names = []
last_names = []

for name in names:
    first_names.append(name[:name.find(' ')])
    last_names.append(name[1+name.find(' '):])

customerDict["first"] = first_names
customerDict["last"] = last_names

customerDF = pandas.DataFrame.from_dict(data=customerDict)
customerDF


,first,last
0,Audrey,Flores
1,Charlotte,Thompson
2,Zoey,Cox
3,Stella,Wood
4,Savannah,Reyes
5,Violet,King
6,Abigail,Scott
7,Bella,Reed
8,Maria,Martin
9,Audrey,Gutierrez


### Store

In [8]:
generateData = "genData.json"

ROOT = os.path.dirname(os.getcwd()) + '\\resource\\' + generateData

storeDict = {}

with open(ROOT, 'r') as file:
    data = json.load(file)
    data = data["Store"]

listAdd = []
for dict in data:
    listAdd.append(data[dict])

    
storeDf = pandas.DataFrame.from_records(listAdd)
storeDf

,Name,Address,City,Texas,Zip Code
0,South Austin Drive,102 South Austin Drive,Austin,Texas,72034
1,Western Drive,111 Western Drive,Houston,Texas,99345
2,North Lake,880 North Lake Drive,Dallas,Texas,14783


### Employee

In [139]:
generateData = "genData.json"

ROOT = os.path.dirname(os.getcwd()) + '\\resource\\' + generateData

with open(ROOT, 'r') as file:
    data = json.load(file)
    data = data["Employee"]

# get each stores employees
stores = []
for store in data:
    stores.append(data[store])


In [142]:
'''
{'one': {'EmployeeID': '1', 'ManagerID': 'Null', 'FirstName': 'Tom', 'LastName': 'Bates', 'StoreID': '1'}, 'two': {
'''


df = pandas.DataFrame(stores[0]).T
stores.pop(0)

for data in stores:
    dfNew = pandas.DataFrame(data).T
    df = pandas.concat([df,dfNew])

df

,EmployeeID,ManagerID,FirstName,LastName,StoreID
one,5,Null,Toby,Harts,2
two,6,5,Ivan,Martin,2
three,7,5,Daniel,Isacc,2
four,8,5,Kelly,Aucoin,2
one,9,Null,Ben,Downey,3
two,10,Null,Arthur,Alan,3
three,11,9,William,Kyber,3
four,12,10,Neil,Tyson,3
five,13,9,Axe,Rod,3
six,14,10,Eli,Brandbury,3


### Product

In [202]:
generateData = "genData.json"

ROOT = os.path.dirname(os.getcwd()) + '\\resource\\' + generateData

with open(ROOT, 'r') as file:
    data = json.load(file)
    data = data["Product"]

# get each stores employees
product = []
for products in data:
    product.append(data[products])


In [203]:
# product
[   ['Hot Cocoa','Eait'], [4.5, 234 ]   ]

[['Hot Cocoa', 'Eait'], [4.5, 234]]

In [204]:
productDF = pandas.DataFrame.from_records(product).T

In [205]:
productDF = productDF.rename(columns={0: 'Product', 1: 'Price'})

In [206]:
productDF

,Product,Price
0,Hot Cocoa,4.5
1,Espresso,3
2,Ristretto,3.5
3,Americano,4
4,Latte,5.5
5,Cappuccino,3.75
6,Macchiato,4.25
7,Cortado,3.25
8,Mocha,4
9,Flat White,3.5


### Order

In [ ]:
OrderID
EmployeeID 
CustomerID 
StoreID 
OrderDate 
TotalAmount

## Part 2

In [ ]:
import PyMySQL